# Tasca M10 T01

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


### - Exercici 1
Realitza web scraping de dues de les tres pàgines web proposades utilitzant BeautifulSoup primer i Selenium després. 

- http://quotes.toscrape.com

- https://www.bolsamadrid.es

- www.wikipedia.es (fes alguna cerca primer i escrapeja algun contingut)

Primero vamos a realizar una request de la página y luego vamos a parsear el contenido con el parseador de html usando BeautifulSoup

In [2]:
url = "https://quotes.toscrape.com/"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

Con la herramienta de inspección vemos que todas las citas están contenidas en contenedores div que pertenecen a la clase 'quote'. Vamos a seleccionarlas.

In [3]:
quotes=soup.find_all("div", class_="quote")

In [4]:
for quote in quotes:
    print(quote, end="\n"*2)

<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="tag" href="/tag/change/page/1/">change</a>
<a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
<a class="tag" href="/tag/thinking/page/1/">thinking</a>
<a class="tag" href="/tag/world/page/1/">world</a>
</div>
</div>

<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>
<span>by <small class="author" itemprop="author">J.K. 

Ya tenemos las citas recogidas en la variable quotes, pero con mucho html y la información mezclada. Vamos a sacar ahora la información que necesitamos: el texto de la cita, el autor y los tags.

In [12]:
for quote in quotes:
    text=quote.find("span", class_="text").text
    author=quote.find("small", class_="author").text
    tag_html=quote.find_all("a", class_="tag")
    tag_list=[]
    for tag in tag_html:
        tag_list.append(tag.text)
    print(text, author, tag_list)

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.” Albert Einstein ['change', 'deep-thoughts', 'thinking', 'world']
“It is our choices, Harry, that show what we truly are, far more than our abilities.” J.K. Rowling ['abilities', 'choices']
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.” Albert Einstein ['inspirational', 'life', 'live', 'miracle', 'miracles']
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.” Jane Austen ['aliteracy', 'books', 'classic', 'humor']
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.” Marilyn Monroe ['be-yourself', 'inspirational']
“Try not to become a man of success. Rather become a man of value.” Albert Einstein ['adulthood', 'success', 'value']
“It is better to be hated for what you are than to b

Crearemos un dataframe y guardaremos los datos en él

In [14]:
df = pd.DataFrame(columns=['text', 'author', 'tags'])

In [16]:
for quote in quotes:
    text=quote.find("span", class_="text").text
    author=quote.find("small", class_="author").text
    tag_html=quote.find_all("a", class_="tag")
    tag_list=[]
    for tag in tag_html:
        tag_list.append(tag.text)
    df = pd.concat([df, pd.DataFrame([{'text': text, 'author':author, 'tags': tag_list}])], ignore_index=True)

In [17]:
df.head()

,text,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"


Como la web tiene 10 páginas y queremos almacenarlas todas, crearemos el mismo proceso que hemos hecho pero dentro de un bucle

In [20]:
df = pd.DataFrame(columns=['text', 'author', 'tags'])

In [21]:
for i in range(1,11):
    url="https://quotes.toscrape.com/page/" + str(i) + "/"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    quotes=soup.find_all("div", class_="quote")
    for quote in quotes:
        text=quote.find("span", class_="text").text
        author=quote.find("small", class_="author").text
        tag_html=quote.find_all("a", class_="tag")
        tag_list=[]
        for tag in tag_html:
            tag_list.append(tag.text)
        df = pd.concat([df, pd.DataFrame([{'text': text, 'author':author, 'tags': tag_list}])], ignore_index=True)
    

In [22]:
df.count()

text      100
author    100
tags      100
dtype: int64

La web tiene 10 citas por página, así que podemos ver que ha guardado el contenido de toda la web en el dataframe. A continuación vamos a almacenar nuestro dataframe en un csv, para poderlo usar en otro momento.

In [23]:
df.to_csv("bs_quotes.csv")